<a href="https://colab.research.google.com/github/wonjinyoo/AutoML/blob/main/CreateML_modelconversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Apple CoreML, CreateML

In [ ]:
!pip install coremltools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf

# tf.keras 이용하여 MobileNet v2 다운로드
keras_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    classes=1000,
    weights="imagenet",
)

# 클래스 레이블 다운로드
import urllib
label_url = "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt"
class_labels = urllib.request.urlopen(label_url).read().splitlines()
class_labels = class_labels[1:]
assert len(class_labels) == 1000

# 클래스 레이블이 모두 문자열이 되도록 한번 더 확인
for i, label in enumerate(class_labels):
  if isinstance(label, bytes):
    class_labels[i] = label.decode("utf-8")

In [ ]:
# 불러온 모델을 CoreML 형식으로 변환
import coremltools as ct

# 입력 데이터 타입을 이미지로 정의
image_input = ct.ImageType(
    shape=(1, 224, 224, 3),
    scale=1/127,
    bias=[-1, -1, -1],
)

# 클래스 레이블 설정
classifier_config = ct.ClassifierConfig(class_labels)

# CoreML 도구가 제공하는 변환 API를 이용해 모델 변환
model = ct.convert(
    keras_model,
    inputs=[image_input],
    classifier_config=classifier_config,
)

Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 89.85 passes/s]


In [ ]:
# CoreML 형식으로 변환된 모델 상세 정보 확인
print(model.get_spec().description)

input {
  name: "input_3"
  type {
    imageType {
      width: 224
      height: 224
      colorSpace: RGB
    }
  }
}
output {
  name: "classLabel"
  type {
    stringType {
    }
  }
}
output {
  name: "classLabel_probs"
  type {
    dictionaryType {
      stringKeyType {
      }
    }
  }
}
predictedFeatureName: "classLabel"
predictedProbabilitiesName: "classLabel_probs"
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "tensorflow==2.15.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "7.2"
  }
}

